# Información de Campos del dataframe

| Field   | Comment                                                                 |
|---------|-------------------------------------------------------------------------|
| START   | The start timestamp of the template execution in milliseconds           |
| END     | The end timestamp of the template execution in milliseconds             |
| TIMEOUT | Indicates if the execution exceeded a predefined time limit             |
| system  | The name of the system used (e.g., PIONIER)                             |
| procname| The process name associated with the template execution                 |
| TPL_ID  | The filename of the corresponding template file                         |
| ERROR   | Indicates if there was an error during execution                        |
| Aborted | Indicates if the template execution was aborted (manually or because an error) |
| SECONDS | The duration of the template execution in seconds                       |
| TEL     | The class of telescope used in the observation, in this dataset it is only AT |

# Información de Campos de Log (metadata de los Log)

| Field       | Description                                                                                           |
|-------------|-------------------------------------------------------------------------------------------------------|
| @timestamp  | The timestamp of the log entry in milliseconds.                                                       |
| system      | The name of the system (e.g., PIONIER) from which the log entry originates.                           |
| hostname    | The hostname of the machine where the log entry was generated.                                        |
| loghost     | The host of the logging system that generated the entry.                                              |
| logtype     | Type of the log entry (e.g., LOG, FEVT, ERR), indicating its nature such as general log, event, or error. |
| envname     | The environment name where the log was generated, providing context for the log entry.                |
| procname    | The name of the process that generated the log entry.                                                 |
| procid      | The process ID associated with the log entry.                                                         |
| module      | The module from which the log entry originated, indicating the specific part of the system.           |
| keywname    | Name of any keyword associated with the log entry, if applicable. It is always paired with keywvalue. |
| keywvalue   | Value of the keyword mentioned in keywname, if applicable.                                            |
| keywmask    | Mask or additional context for the keyword, if applicable.                                            |
| logtext     | The actual text of the log entry, providing detailed information about the event or action.           |
| trace_id    | A unique identifier associated with each log entry, corresponds to id in metadata table.              |

In [1]:
import os
import pandas as pd
import subprocess
from urllib.request import urlretrieve

REPO_URL = 'https://huggingface.co/datasets/Paranal/parlogs-observations/resolve/main/data'

if 'COLAB_RELEASE_TAG' in os.environ.keys():
    PATH = 'data/'  # Convenient name to be Colab compatible
else:
    PATH = 'D:/Ufro/11 nivel/Lab modelacion 2/Trabajo_paramal/collaborations-UFRO-2024-2s/data'  # Local directory to your system

def load_dataset(INSTRUMENT, RANGE):
    fname = f'{INSTRUMENT}-{RANGE}-traces.parket'
    if not os.path.exists(f'{PATH}/{fname}'):
        urlretrieve(f'{REPO_URL}/{fname}', f'{PATH}/{fname}')
    df_inst = pd.read_parquet(f'{PATH}/{fname}')

    fname = f'{INSTRUMENT}-{RANGE}-traces-SUBSYSTEMS.parket'
    if not os.path.exists(f'{PATH}/{fname}'):
        urlretrieve(f'{REPO_URL}/{fname}', f'{PATH}/{fname}')
    df_subs = pd.read_parquet(f'{PATH}/{fname}')

    fname = f'{INSTRUMENT}-{RANGE}-traces-TELESCOPES.parket'
    if not os.path.exists(f'{PATH}/{fname}'):
        urlretrieve(f'{REPO_URL}/{fname}', f'{PATH}/{fname}')
    df_tele = pd.read_parquet(f'{PATH}/{fname}')

    all_traces = [df_inst, df_subs, df_tele]

    df_all = pd.concat(all_traces)
    df_all.sort_values('@timestamp', inplace=True)
    df_all.reset_index(drop=True, inplace=True)

    return df_all

def load_trace(INSTRUMENT, RANGE, trace_id):
    df_all = load_dataset(INSTRUMENT, RANGE)
    df_all = df_all[df_all['trace_id'] == trace_id]
    return df_all

def load_meta(INSTRUMENT, RANGE):
    fname = f'{INSTRUMENT}-{RANGE}-meta.parket'
    if not os.path.exists(f'{PATH}/{fname}'):
        urlretrieve(f'{REPO_URL}/{fname}', f'{PATH}/{fname}')
    df_meta = pd.read_parquet(f'{PATH}/{fname}')
    return df_meta

# Función para generar el código PlantUML para un solo usuario
def generate_plantuml_code_single_user(meta_row, trace_row):
    uml_code = "@startuml\n"
    uml_code += "actor User\n"
    uml_code += "participant System\n"
    uml_code += "participant Instrument\n"
    uml_code += "participant Subsystem\n\n"

    start_time = meta_row['START']
    end_time = meta_row['END']
    procname = meta_row['procname']
    status = 'success' if not meta_row['ERROR'] else 'failure'
    logtext = trace_row['logtext']

    uml_code += f"User -> System: Send Command ({procname})\n"
    uml_code += "activate System\n"
    uml_code += f"System -> Instrument: Forward Command ({procname})\n"
    uml_code += "activate Instrument\n"
    uml_code += f"Instrument -> Subsystem: Execute Command ({procname})\n"
    uml_code += "activate Subsystem\n\n"

    if status == 'success':
        uml_code += f"Subsystem -> Instrument: Return Success ({start_time}, {end_time})\n"
    else:
        uml_code += f"Subsystem -> Instrument: Return Failure ({start_time}, {end_time})\n"
    
    uml_code += "deactivate Subsystem\n\n"
    uml_code += f"Instrument -> System: Return Status ({status})\n"
    uml_code += "deactivate Instrument\n\n"
    uml_code += f"System -> User: Return Status ({status})\n"
    uml_code += "deactivate System\n\n"
    uml_code += f"note right of System: Log Entry - {logtext}\n"

    uml_code += "@enduml"
    return uml_code

# Cargar los datos de observación y metadatos
INSTRUMENT = "PIONIER"
RANGE = "1w"
TRACE_ID = 100  # Ejemplo de trace_id, ajusta según tus datos

df_meta = load_meta(INSTRUMENT, RANGE)
df_trace = load_trace(INSTRUMENT, RANGE, TRACE_ID)

# Verificar las columnas de los DataFrames
print("Columnas en df_meta:", df_meta.columns)
print("Columnas en df_trace:", df_trace.columns)

# Seleccionar una fila del DataFrame para el usuario

meta_row=df_meta.iloc[TRACE_ID]


if 'trace_id' in df_trace.columns:
    trace_row = df_trace.iloc[0]
else:
    print("La columna 'trace_id' no está presente en df_trace. Verifica el nombre de la columna.")

# Generar el código PlantUML para un solo usuario
uml_code = generate_plantuml_code_single_user(meta_row, trace_row)

# Guardar el código PlantUML en un archivo
with open('dynamic_diagram.puml', 'w') as file:
    file.write(uml_code)

# Renderizar el diagrama usando PlantUML a través de subprocess
subprocess.run(['java', '-jar', 'plantuml.jar', 'dynamic_diagram.puml'])

Columnas en df_meta: Index(['START', 'END', 'TIMEOUT', 'system', 'procname', 'TPL_ID', 'ERROR',
       'Aborted', 'SECONDS', 'TEL'],
      dtype='object')
Columnas en df_trace: Index(['@timestamp', 'system', 'hostname', 'loghost', 'logtype', 'envname',
       'procname', 'procid', 'module', 'keywname', 'keywvalue', 'keywmask',
       'logtext', 'trace_id'],
      dtype='object')


CompletedProcess(args=['java', '-jar', 'plantuml.jar', 'dynamic_diagram.puml'], returncode=1)

In [2]:
from PIL import Image

In [3]:
df_meta.iloc[TRACE_ID]

START       2019-04-03 17:08:08.894000
END         2019-04-03 17:08:42.030000
TIMEOUT                          False
system                         PIONIER
procname                       bob_ins
TPL_ID         PIONIER_gen_tec_standby
ERROR                            False
Aborted                          False
SECONDS                           33.0
TEL                                 AT
Name: 100, dtype: object

In [4]:
trace_id=10

In [5]:
import os
import pandas as pd
import subprocess
from urllib.request import urlretrieve

REPO_URL = 'https://huggingface.co/datasets/Paranal/parlogs-observations/resolve/main/data'

if 'COLAB_RELEASE_TAG' in os.environ.keys():
    PATH = 'data/'  # Convenient name to be Colab compatible
else:
    PATH = 'D:/Ufro/11 nivel/Lab modelacion 2/Trabajo_paramal/collaborations-UFRO-2024-2s/data'  # Local directory to your system

def load_dataset(INSTRUMENT, RANGE):
    fname = f'{INSTRUMENT}-{RANGE}-traces.parket'
    if not os.path.exists(f'{PATH}/{fname}'):
        urlretrieve(f'{REPO_URL}/{fname}', f'{PATH}/{fname}')
    df_inst = pd.read_parquet(f'{PATH}/{fname}')

    fname = f'{INSTRUMENT}-{RANGE}-traces-SUBSYSTEMS.parket'
    if not os.path.exists(f'{PATH}/{fname}'):
        urlretrieve(f'{REPO_URL}/{fname}', f'{PATH}/{fname}')
    df_subs = pd.read_parquet(f'{PATH}/{fname}')

    fname = f'{INSTRUMENT}-{RANGE}-traces-TELESCOPES.parket'
    if not os.path.exists(f'{PATH}/{fname}'):
        urlretrieve(f'{REPO_URL}/{fname}', f'{PATH}/{fname}')
    df_tele = pd.read_parquet(f'{PATH}/{fname}')

    all_traces = [df_inst, df_subs, df_tele]

    df_all = pd.concat(all_traces)
    df_all.sort_values('@timestamp', inplace=True)
    df_all.reset_index(drop=True, inplace=True)

    return df_all

def load_trace(INSTRUMENT, RANGE, trace_id):
    df_all = load_dataset(INSTRUMENT, RANGE)
    df_all = df_all[df_all['trace_id'] == trace_id]
    return df_all

def load_meta(INSTRUMENT, RANGE):
    fname = f'{INSTRUMENT}-{RANGE}-meta.parket'
    if not os.path.exists(f'{PATH}/{fname}'):
        urlretrieve(f'{REPO_URL}/{fname}', f'{PATH}/{fname}')
    df_meta = pd.read_parquet(f'{PATH}/{fname}')
    return df_meta

# Función para generar el código PlantUML para un solo usuario
def generate_plantuml_code_single_user(meta_row, trace_row):
    uml_code = "@startuml\n"
    uml_code += "actor User\n"
    uml_code += "participant System\n"
    uml_code += "participant Instrument\n"
    uml_code += "participant Subsystem\n\n"

    start_time = meta_row['START']
    end_time = meta_row['END']
    procname = meta_row['procname']
    status = 'success' if not meta_row['ERROR'] else 'failure'
    logtext = trace_row['logtext']

    uml_code += f"User -> System: Send Command ({procname})\n"
    uml_code += "activate System\n"
    uml_code += f"System -> Instrument: Forward Command ({procname})\n"
    uml_code += "activate Instrument\n"
    uml_code += f"Instrument -> Subsystem: Execute Command ({procname})\n"
    uml_code += "activate Subsystem\n\n"

    if status == 'success':
        uml_code += f"Subsystem -> Instrument: Return Success ({start_time}, {end_time})\n"
    else:
        uml_code += f"Subsystem -> Instrument: Return Failure ({start_time}, {end_time})\n"
    
    uml_code += "deactivate Subsystem\n\n"
    uml_code += f"Instrument -> System: Return Status ({status})\n"
    uml_code += "deactivate Instrument\n\n"
    uml_code += f"System -> User: Return Status ({status})\n"
    uml_code += "deactivate System\n\n"
    uml_code += f"note right of System: Log Entry - {logtext}\n"

    uml_code += "@enduml"
    return uml_code

# Cargar los datos de observación y metadatos
INSTRUMENT = "PIONIER"
RANGE = "1w"
TRACE_ID = 100  # Ejemplo de trace_id, ajusta según tus datos

df_meta = load_meta(INSTRUMENT, RANGE)
df_trace = load_trace(INSTRUMENT, RANGE, TRACE_ID)

# Verificar las columnas de los DataFrames
print("Columnas en df_meta:", df_meta.columns)
print("Columnas en df_trace:", df_trace.columns)

# Seleccionar una fila del DataFrame para el usuario
meta_row = df_meta.iloc[TRACE_ID]

if 'trace_id' in df_trace.columns:
    trace_row = df_trace.iloc[0]
else:
    print("La columna 'trace_id' no está presente en df_trace. Verifica el nombre de la columna.")

# Generar el código PlantUML para un solo usuario
uml_code = generate_plantuml_code_single_user(meta_row, trace_row)

# Guardar el código PlantUML en un archivo
with open('dynamic_diagram.puml', 'w') as file:
    file.write(uml_code)

# Renderizar el diagrama usando PlantUML a través de subprocess
subprocess.run(['java', '-jar', 'plantuml.jar', 'dynamic_diagram.puml'])

Columnas en df_meta: Index(['START', 'END', 'TIMEOUT', 'system', 'procname', 'TPL_ID', 'ERROR',
       'Aborted', 'SECONDS', 'TEL'],
      dtype='object')
Columnas en df_trace: Index(['@timestamp', 'system', 'hostname', 'loghost', 'logtype', 'envname',
       'procname', 'procid', 'module', 'keywname', 'keywvalue', 'keywmask',
       'logtext', 'trace_id'],
      dtype='object')


CompletedProcess(args=['java', '-jar', 'plantuml.jar', 'dynamic_diagram.puml'], returncode=1)

In [6]:
df_trace = load_trace("MATISSE", "1w", 0)
df_trace = df_trace[['@timestamp', 'system', 'procname', 'logtext', 'trace_id']]
df_tratamiento=df_trace[ df_trace['logtext'].str.contains("CHKMOON") ][:]
lista = ['Send','Received command', 'Waiting', 'Succesfully completed', 'Last reply']
Lista_system = []

for x in lista:
    df_cc = df_tratamiento[df_tratamiento['logtext'].str.contains(x)]['system']
    Lista_system.extend(df_cc.tolist())

print(Lista_system)

lista1 = ['Send','Received command', 'Waiting', 'Succesfully completed', 'Last reply']

Lista_system = []

for x in lista1:
    df_cc = df_tratamiento[df_tratamiento['logtext'].str.contains(x)]['procname']
    Lista_system.extend(df_cc.tolist())

print(Lista_system)


['MATISSE', 'ISS', 'MATISSE', 'ISS', 'MATISSE']
['bob_ins', 'issifControl', 'bob_ins', 'issifControl', 'bob_ins']


In [7]:
import os
import pandas as pd
import subprocess
from urllib.request import urlretrieve

REPO_URL = 'https://huggingface.co/datasets/Paranal/parlogs-observations/resolve/main/data'

if 'COLAB_RELEASE_TAG' in os.environ.keys():
    PATH = 'data/'  # Convenient name to be Colab compatible
else:
    PATH = 'D:/Ufro/11 nivel/Lab modelacion 2/Trabajo_paramal/collaborations-UFRO-2024-2s/data'  # Local directory to your system

def load_dataset(INSTRUMENT, RANGE):
    fname = f'{INSTRUMENT}-{RANGE}-traces.parket'
    if not os.path.exists(f'{PATH}/{fname}'):
        urlretrieve(f'{REPO_URL}/{fname}', f'{PATH}/{fname}')
    df_inst = pd.read_parquet(f'{PATH}/{fname}')

    fname = f'{INSTRUMENT}-{RANGE}-traces-SUBSYSTEMS.parket'
    if not os.path.exists(f'{PATH}/{fname}'):
        urlretrieve(f'{REPO_URL}/{fname}', f'{PATH}/{fname}')
    df_subs = pd.read_parquet(f'{PATH}/{fname}')

    fname = f'{INSTRUMENT}-{RANGE}-traces-TELESCOPES.parket'
    if not os.path.exists(f'{PATH}/{fname}'):
        urlretrieve(f'{REPO_URL}/{fname}', f'{PATH}/{fname}')
    df_tele = pd.read_parquet(f'{PATH}/{fname}')

    all_traces = [df_inst, df_subs, df_tele]

    df_all = pd.concat(all_traces)
    df_all.sort_values('@timestamp', inplace=True)
    df_all.reset_index(drop=True, inplace=True)

    return df_all

def load_trace(INSTRUMENT, RANGE, trace_id):
    df_all = load_dataset(INSTRUMENT, RANGE)
    df_all = df_all[df_all['trace_id'] == trace_id]
    return df_all

def load_meta(INSTRUMENT, RANGE):
    fname = f'{INSTRUMENT}-{RANGE}-meta.parket'
    if not os.path.exists(f'{PATH}/{fname}'):
        urlretrieve(f'{REPO_URL}/{fname}', f'{PATH}/{fname}')
    df_meta = pd.read_parquet(f'{PATH}/{fname}')
    return df_meta

# Función para generar el código PlantUML para un solo usuario
def generate_plantuml_code_single_user(meta_row, trace_row, Lista_system, Lista_procname):
    uml_code = "@startuml\n"
    uml_code += "actor User\n"
    uml_code += "participant System\n"
    uml_code += "participant Instrument\n"
    uml_code += "participant Subsystem\n\n"

    start_time = meta_row['START']
    end_time = meta_row['END']
    procname = meta_row['procname']
    status = 'success' if not meta_row['ERROR'] else 'failure'
    logtext = trace_row['logtext']

    uml_code += f"User -> System: Send Command ({procname})\n"
    uml_code += "activate System\n"

    for system, procname in zip(Lista_system, Lista_procname):
        uml_code += f"System -> {system}: Forward Command ({procname})\n"
        uml_code += f"activate {system}\n"
        uml_code += f"{system} -> Instrument: Forward Command ({procname})\n"
        uml_code += "activate Instrument\n"
        uml_code += f"Instrument -> Subsystem: Execute Command ({procname})\n"
        uml_code += "activate Subsystem\n\n"

        if status == 'success':
            uml_code += f"Subsystem -> Instrument: Return Success ({start_time}, {end_time})\n"
        else:
            uml_code += f"Subsystem -> Instrument: Return Failure ({start_time}, {end_time})\n"
        
        uml_code += "deactivate Subsystem\n\n"
        uml_code += f"Instrument -> {system}: Return Status ({status})\n"
        uml_code += "deactivate Instrument\n\n"
        uml_code += f"{system} -> System: Return Status ({status})\n"
        uml_code += f"deactivate {system}\n\n"
        uml_code += f"note right of {system}: Log Entry - {logtext}\n"

    uml_code += f"System -> User: Return Status ({status})\n"
    uml_code += "deactivate System\n\n"
    uml_code += f"note right of System: Log Entry - {logtext}\n"

    uml_code += "@enduml"
    return uml_code

# Cargar los datos de observación y metadatos
INSTRUMENT = "MATISSE"
RANGE = "1w"
TRACE_ID = 0  # Ejemplo de trace_id, ajusta según tus datos

df_meta = load_meta(INSTRUMENT, RANGE)
df_trace = load_trace(INSTRUMENT, RANGE, TRACE_ID)

# Verificar las columnas de los DataFrames
print("Columnas en df_meta:", df_meta.columns)
print("Columnas en df_trace:", df_trace.columns)

# Seleccionar una fila del DataFrame para el usuario
meta_row = df_meta.iloc[TRACE_ID]

if 'trace_id' in df_trace.columns:
    trace_row = df_trace.iloc[0]
else:
    print("La columna 'trace_id' no está presente en df_trace. Verifica el nombre de la columna.")

# Filtrar y procesar los datos de df_trace
df_trace = load_trace("MATISSE", "1w", 0)
df_trace = df_trace[['@timestamp', 'system', 'procname', 'logtext', 'trace_id']]
df_tratamiento = df_trace[df_trace['logtext'].str.contains("CHKMOON")][:]

lista = ['Send', 'Received command', 'Waiting', 'Succesfully completed', 'Last reply']
Lista_system = []

for x in lista:
    df_cc = df_tratamiento[df_tratamiento['logtext'].str.contains(x)]['system']
    Lista_system.extend(df_cc.tolist())

print(Lista_system)

lista1 = ['Send', 'Received command', 'Waiting', 'Succesfully completed', 'Last reply']
Lista_procname = []

for x in lista1:
    df_cc = df_tratamiento[df_tratamiento['logtext'].str.contains(x)]['procname']
    Lista_procname.extend(df_cc.tolist())

print(Lista_procname)

# Generar el código PlantUML para un solo usuario
uml_code = generate_plantuml_code_single_user(meta_row, trace_row, Lista_system, Lista_procname)

# Guardar el código PlantUML en un archivo
with open('dynamic_diagram.puml', 'w') as file:
    file.write(uml_code)

# Renderizar el diagrama usando PlantUML a través de subprocess
subprocess.run(['java', '-jar', 'plantuml.jar', 'dynamic_diagram.puml'])

Columnas en df_meta: Index(['START', 'END', 'TIMEOUT', 'system', 'procname', 'TPL_ID', 'ERROR',
       'Aborted', 'SECONDS', 'TEL'],
      dtype='object')
Columnas en df_trace: Index(['@timestamp', 'system', 'hostname', 'loghost', 'logtype', 'envname',
       'procname', 'procid', 'module', 'keywname', 'keywvalue', 'keywmask',
       'logtext', 'trace_id'],
      dtype='object')
['MATISSE', 'ISS', 'MATISSE', 'ISS', 'MATISSE']
['bob_ins', 'issifControl', 'bob_ins', 'issifControl', 'bob_ins']


CompletedProcess(args=['java', '-jar', 'plantuml.jar', 'dynamic_diagram.puml'], returncode=1)

In [8]:
import os
import pandas as pd
import subprocess
from urllib.request import urlretrieve

REPO_URL = 'https://huggingface.co/datasets/Paranal/parlogs-observations/resolve/main/data'

if 'COLAB_RELEASE_TAG' in os.environ.keys():
    PATH = 'data/'  # Convenient name to be Colab compatible
else:
    PATH = 'D:/Ufro/11 nivel/Lab modelacion 2/Trabajo_paramal/collaborations-UFRO-2024-2s/data'  # Local directory to your system

def load_dataset(INSTRUMENT, RANGE):
    fname = f'{INSTRUMENT}-{RANGE}-traces.parket'
    if not os.path.exists(f'{PATH}/{fname}'):
        urlretrieve(f'{REPO_URL}/{fname}', f'{PATH}/{fname}')
    df_inst = pd.read_parquet(f'{PATH}/{fname}')

    fname = f'{INSTRUMENT}-{RANGE}-traces-SUBSYSTEMS.parket'
    if not os.path.exists(f'{PATH}/{fname}'):
        urlretrieve(f'{REPO_URL}/{fname}', f'{PATH}/{fname}')
    df_subs = pd.read_parquet(f'{PATH}/{fname}')

    fname = f'{INSTRUMENT}-{RANGE}-traces-TELESCOPES.parket'
    if not os.path.exists(f'{PATH}/{fname}'):
        urlretrieve(f'{REPO_URL}/{fname}', f'{PATH}/{fname}')
    df_tele = pd.read_parquet(f'{PATH}/{fname}')

    all_traces = [df_inst, df_subs, df_tele]

    df_all = pd.concat(all_traces)
    df_all.sort_values('@timestamp', inplace=True)
    df_all.reset_index(drop=True, inplace=True)

    return df_all

def load_trace(INSTRUMENT, RANGE, trace_id):
    df_all = load_dataset(INSTRUMENT, RANGE)
    df_all = df_all[df_all['trace_id'] == trace_id]
    return df_all

def load_meta(INSTRUMENT, RANGE):
    fname = f'{INSTRUMENT}-{RANGE}-meta.parket'
    if not os.path.exists(f'{PATH}/{fname}'):
        urlretrieve(f'{REPO_URL}/{fname}', f'{PATH}/{fname}')
    df_meta = pd.read_parquet(f'{PATH}/{fname}')
    return df_meta

# Función para generar el código PlantUML para un solo usuario
def generate_plantuml_code_single_user(meta_row, trace_row, Lista_system, Lista_procname):
    uml_code = "@startuml\n"
    uml_code += "actor User\n"
    uml_code += "participant System\n"
    uml_code += "participant Instrument\n"
    uml_code += "participant Subsystem\n\n"

    start_time = meta_row['START']
    end_time = meta_row['END']
    procname = meta_row['procname']
    status = 'success' if not meta_row['ERROR'] else 'failure'
    logtext = trace_row['logtext']

    uml_code += f"User -> System: Send Command ({procname})\n"
    uml_code += "activate System\n"

    previous_procname = None
    for system, procname in zip(Lista_system, Lista_procname):
        if procname != previous_procname:
            uml_code += f"note right of System: procname changed to {procname}\n"
            previous_procname = procname

        uml_code += f"System -> {system}: Forward Command ({procname})\n"
        uml_code += f"activate {system}\n"
        uml_code += f"{system} -> Instrument: Forward Command ({procname})\n"
        uml_code += "activate Instrument\n"
        uml_code += f"Instrument -> Subsystem: Execute Command ({procname})\n"
        uml_code += "activate Subsystem\n\n"

        if status == 'success':
            uml_code += f"Subsystem -> Instrument: Return Success ({start_time}, {end_time})\n"
        else:
            uml_code += f"Subsystem -> Instrument: Return Failure ({start_time}, {end_time})\n"
        
        uml_code += "deactivate Subsystem\n\n"
        uml_code += f"Instrument -> {system}: Return Status ({status})\n"
        uml_code += "deactivate Instrument\n\n"
        uml_code += f"{system} -> System: Return Status ({status})\n"
        uml_code += f"deactivate {system}\n\n"
        uml_code += f"note right of {system}: Log Entry - {logtext}\n"

    uml_code += f"System -> User: Return Status ({status})\n"
    uml_code += "deactivate System\n\n"
    uml_code += f"note right of System: Log Entry - {logtext}\n"

    uml_code += "@enduml"
    return uml_code

# Cargar los datos de observación y metadatos
INSTRUMENT = "MATISSE"
RANGE = "1w"
TRACE_ID = 0  # Ejemplo de trace_id, ajusta según tus datos

df_meta = load_meta(INSTRUMENT, RANGE)
df_trace = load_trace(INSTRUMENT, RANGE, TRACE_ID)

# Verificar las columnas de los DataFrames
print("Columnas en df_meta:", df_meta.columns)
print("Columnas en df_trace:", df_trace.columns)

# Seleccionar una fila del DataFrame para el usuario
meta_row = df_meta.iloc[TRACE_ID]

if 'trace_id' in df_trace.columns:
    trace_row = df_trace.iloc[0]
else:
    print("La columna 'trace_id' no está presente en df_trace. Verifica el nombre de la columna.")

# Filtrar y procesar los datos de df_trace
df_trace = load_trace("MATISSE", "1w", 0)
df_trace = df_trace[['@timestamp', 'system', 'procname', 'logtext', 'trace_id']]
df_tratamiento = df_trace[df_trace['logtext'].str.contains("CHKMOON")][:]

lista = ['Send', 'Received command', 'Waiting', 'Succesfully completed', 'Last reply']
Lista_system = []

for x in lista:
    df_cc = df_tratamiento[df_tratamiento['logtext'].str.contains(x)]['system']
    Lista_system.extend(df_cc.tolist())

print(Lista_system)

lista1 = ['Send', 'Received command', 'Waiting', 'Succesfully completed', 'Last reply']
Lista_procname = []

for x in lista1:
    df_cc = df_tratamiento[df_tratamiento['logtext'].str.contains(x)]['procname']
    Lista_procname.extend(df_cc.tolist())

print(Lista_procname)

# Generar el código PlantUML para un solo usuario
uml_code = generate_plantuml_code_single_user(meta_row, trace_row, Lista_system, Lista_procname)

# Guardar el código PlantUML en un archivo
with open('dynamic_diagram.puml', 'w') as file:
    file.write(uml_code)

# Renderizar el diagrama usando PlantUML a través de subprocess
subprocess.run(['java', '-jar', 'plantuml.jar', 'dynamic_diagram.puml'])

Columnas en df_meta: Index(['START', 'END', 'TIMEOUT', 'system', 'procname', 'TPL_ID', 'ERROR',
       'Aborted', 'SECONDS', 'TEL'],
      dtype='object')
Columnas en df_trace: Index(['@timestamp', 'system', 'hostname', 'loghost', 'logtype', 'envname',
       'procname', 'procid', 'module', 'keywname', 'keywvalue', 'keywmask',
       'logtext', 'trace_id'],
      dtype='object')
['MATISSE', 'ISS', 'MATISSE', 'ISS', 'MATISSE']
['bob_ins', 'issifControl', 'bob_ins', 'issifControl', 'bob_ins']


CompletedProcess(args=['java', '-jar', 'plantuml.jar', 'dynamic_diagram.puml'], returncode=1)

In [9]:
import os
import pandas as pd
import subprocess
from urllib.request import urlretrieve

REPO_URL = 'https://huggingface.co/datasets/Paranal/parlogs-observations/resolve/main/data'

if 'COLAB_RELEASE_TAG' in os.environ.keys():
    PATH = 'data/'  # Convenient name to be Colab compatible
else:
    PATH = 'D:/Ufro/11 nivel/Lab modelacion 2/Trabajo_paramal/collaborations-UFRO-2024-2s/data'  # Local directory to your system

def load_dataset(INSTRUMENT, RANGE):
    fname = f'{INSTRUMENT}-{RANGE}-traces.parket'
    if not os.path.exists(f'{PATH}/{fname}'):
        urlretrieve(f'{REPO_URL}/{fname}', f'{PATH}/{fname}')
    df_inst = pd.read_parquet(f'{PATH}/{fname}')

    fname = f'{INSTRUMENT}-{RANGE}-traces-SUBSYSTEMS.parket'
    if not os.path.exists(f'{PATH}/{fname}'):
        urlretrieve(f'{REPO_URL}/{fname}', f'{PATH}/{fname}')
    df_subs = pd.read_parquet(f'{PATH}/{fname}')

    fname = f'{INSTRUMENT}-{RANGE}-traces-TELESCOPES.parket'
    if not os.path.exists(f'{PATH}/{fname}'):
        urlretrieve(f'{REPO_URL}/{fname}', f'{PATH}/{fname}')
    df_tele = pd.read_parquet(f'{PATH}/{fname}')

    all_traces = [df_inst, df_subs, df_tele]

    df_all = pd.concat(all_traces)
    df_all.sort_values('@timestamp', inplace=True)
    df_all.reset_index(drop=True, inplace=True)

    return df_all

def load_trace(INSTRUMENT, RANGE, trace_id):
    df_all = load_dataset(INSTRUMENT, RANGE)
    df_all = df_all[df_all['trace_id'] == trace_id]
    return df_all

def load_meta(INSTRUMENT, RANGE):
    fname = f'{INSTRUMENT}-{RANGE}-meta.parket'
    if not os.path.exists(f'{PATH}/{fname}'):
        urlretrieve(f'{REPO_URL}/{fname}', f'{PATH}/{fname}')
    df_meta = pd.read_parquet(f'{PATH}/{fname}')
    return df_meta

# Función para generar el código PlantUML para un solo usuario
def generate_plantuml_code_single_user(meta_row, trace_row, Lista_system, Lista_procname):
    uml_code = "@startuml\n"
    uml_code += "actor User\n"
    uml_code += "participant System\n"
    uml_code += "participant Instrument\n"
    uml_code += "participant Subsystem\n\n"

    start_time = meta_row['START']
    end_time = meta_row['END']
    procname = meta_row['procname']
    status = 'success' if not meta_row['ERROR'] else 'failure'
    logtext = trace_row['logtext']

    uml_code += f"User -> System: Send Command ({procname})\n"
    uml_code += "activate System\n"

    previous_procname = None
    for system, procname in zip(Lista_system, Lista_procname):
        if procname != previous_procname:
            uml_code += f"note right of System: procname changed to {procname}\n"
            previous_procname = procname

        system_name = "MATISSE" if system == "MATISSE" else "ISS"
        uml_code += f"System -> {system_name}: Forward Command ({procname})\n"
        uml_code += f"activate {system_name}\n"
        uml_code += f"{system_name} -> Instrument: Forward Command ({procname})\n"
        uml_code += "activate Instrument\n"
        uml_code += f"Instrument -> Subsystem: Execute Command ({procname})\n"
        uml_code += "activate Subsystem\n\n"

        if status == 'success':
            uml_code += f"Subsystem -> Instrument: Return Success ({start_time}, {end_time})\n"
        else:
            uml_code += f"Subsystem -> Instrument: Return Failure ({start_time}, {end_time})\n"
        
        uml_code += "deactivate Subsystem\n\n"
        uml_code += f"Instrument -> {system_name}: Return Status ({status})\n"
        uml_code += "deactivate Instrument\n\n"
        uml_code += f"{system_name} -> System: Return Status ({status})\n"
        uml_code += f"deactivate {system_name}\n\n"
        uml_code += f"note right of {system_name}: Log Entry - {logtext}\n"

    uml_code += f"System -> User: Return Status ({status})\n"
    uml_code += "deactivate System\n\n"
    uml_code += f"note right of System: Log Entry - {logtext}\n"

    uml_code += "@enduml"
    return uml_code

# Cargar los datos de observación y metadatos
INSTRUMENT = "MATISSE"
RANGE = "1w"
TRACE_ID = 0  # Ejemplo de trace_id, ajusta según tus datos

df_meta = load_meta(INSTRUMENT, RANGE)
df_trace = load_trace(INSTRUMENT, RANGE, TRACE_ID)

# Verificar las columnas de los DataFrames
print("Columnas en df_meta:", df_meta.columns)
print("Columnas en df_trace:", df_trace.columns)

# Seleccionar una fila del DataFrame para el usuario
meta_row = df_meta.iloc[TRACE_ID]

if 'trace_id' in df_trace.columns:
    trace_row = df_trace.iloc[0]
else:
    print("La columna 'trace_id' no está presente en df_trace. Verifica el nombre de la columna.")

# Filtrar y procesar los datos de df_trace
df_trace = load_trace("MATISSE", "1w", 0)
df_trace = df_trace[['@timestamp', 'system', 'procname', 'logtext', 'trace_id']]
df_tratamiento = df_trace[df_trace['logtext'].str.contains("CHKMOON")][:]

lista = ['Send', 'Received command', 'Waiting', 'Succesfully completed', 'Last reply']
Lista_system = []

for x in lista:
    df_cc = df_tratamiento[df_tratamiento['logtext'].str.contains(x)]['system']
    Lista_system.extend(df_cc.tolist())

print(Lista_system)

lista1 = ['Send', 'Received command', 'Waiting', 'Succesfully completed', 'Last reply']
Lista_procname = []

for x in lista1:
    df_cc = df_tratamiento[df_tratamiento['logtext'].str.contains(x)]['procname']
    Lista_procname.extend(df_cc.tolist())

print(Lista_procname)

# Generar el código PlantUML para un solo usuario
uml_code = generate_plantuml_code_single_user(meta_row, trace_row, Lista_system, Lista_procname)

# Guardar el código PlantUML en un archivo
with open('dynamic_diagram.puml', 'w') as file:
    file.write(uml_code)

# Renderizar el diagrama usando PlantUML a través de subprocess
subprocess.run(['java', '-jar', 'plantuml.jar', 'dynamic_diagram.puml'])

Columnas en df_meta: Index(['START', 'END', 'TIMEOUT', 'system', 'procname', 'TPL_ID', 'ERROR',
       'Aborted', 'SECONDS', 'TEL'],
      dtype='object')
Columnas en df_trace: Index(['@timestamp', 'system', 'hostname', 'loghost', 'logtype', 'envname',
       'procname', 'procid', 'module', 'keywname', 'keywvalue', 'keywmask',
       'logtext', 'trace_id'],
      dtype='object')
['MATISSE', 'ISS', 'MATISSE', 'ISS', 'MATISSE']
['bob_ins', 'issifControl', 'bob_ins', 'issifControl', 'bob_ins']


CompletedProcess(args=['java', '-jar', 'plantuml.jar', 'dynamic_diagram.puml'], returncode=1)

In [10]:
import os
import pandas as pd
import subprocess
from urllib.request import urlretrieve

REPO_URL = 'https://huggingface.co/datasets/Paranal/parlogs-observations/resolve/main/data'

if 'COLAB_RELEASE_TAG' in os.environ.keys():
    PATH = 'data/'  # Convenient name to be Colab compatible
else:
    PATH = 'D:/Ufro/11 nivel/Lab modelacion 2/Trabajo_paramal/collaborations-UFRO-2024-2s/data'  # Local directory to your system

def load_dataset(INSTRUMENT, RANGE):
    fname = f'{INSTRUMENT}-{RANGE}-traces.parket'
    if not os.path.exists(f'{PATH}/{fname}'):
        urlretrieve(f'{REPO_URL}/{fname}', f'{PATH}/{fname}')
    df_inst = pd.read_parquet(f'{PATH}/{fname}')

    fname = f'{INSTRUMENT}-{RANGE}-traces-SUBSYSTEMS.parket'
    if not os.path.exists(f'{PATH}/{fname}'):
        urlretrieve(f'{REPO_URL}/{fname}', f'{PATH}/{fname}')
    df_subs = pd.read_parquet(f'{PATH}/{fname}')

    fname = f'{INSTRUMENT}-{RANGE}-traces-TELESCOPES.parket'
    if not os.path.exists(f'{PATH}/{fname}'):
        urlretrieve(f'{REPO_URL}/{fname}', f'{PATH}/{fname}')
    df_tele = pd.read_parquet(f'{PATH}/{fname}')

    all_traces = [df_inst, df_subs, df_tele]

    df_all = pd.concat(all_traces)
    df_all.sort_values('@timestamp', inplace=True)
    df_all.reset_index(drop=True, inplace=True)

    return df_all

def load_trace(INSTRUMENT, RANGE, trace_id):
    df_all = load_dataset(INSTRUMENT, RANGE)
    df_all = df_all[df_all['trace_id'] == trace_id]
    return df_all

def load_meta(INSTRUMENT, RANGE):
    fname = f'{INSTRUMENT}-{RANGE}-meta.parket'
    if not os.path.exists(f'{PATH}/{fname}'):
        urlretrieve(f'{REPO_URL}/{fname}', f'{PATH}/{fname}')
    df_meta = pd.read_parquet(f'{PATH}/{fname}')
    return df_meta

# Función para generar el código PlantUML para un solo usuario
def generate_plantuml_code_single_user(meta_row, trace_row, Lista_system, Lista_procname):
    uml_code = "@startuml\n"
    uml_code += "actor User\n"
    uml_code += "participant System\n"
    uml_code += "participant Instrument\n"
    uml_code += "participant Subsystem\n\n"

    start_time = meta_row['START']
    end_time = meta_row['END']
    procname = meta_row['procname']
    status = 'success' if not meta_row['ERROR'] else 'failure'
    logtext = trace_row['logtext']

    uml_code += f"User -> System: Send Command ({procname})\n"
    uml_code += "activate System\n"

    previous_procname = None
    for system, procname in zip(Lista_system, Lista_procname):
        if procname != previous_procname:
            # Obtener el tiempo de inicio del procname
            proc_start_time = trace_row['@timestamp']
            uml_code += f"note right of System: procname changed to {procname} (Start Time: {proc_start_time})\n"
            previous_procname = procname

        system_name = "MATISSE" if system == "MATISSE" else "ISS"
        uml_code += f"System -> {system_name}: Forward Command ({procname})\n"
        uml_code += f"activate {system_name}\n"
        uml_code += f"{system_name} -> Instrument: Forward Command ({procname})\n"
        uml_code += "activate Instrument\n"
        uml_code += f"Instrument -> Subsystem: Execute Command ({procname})\n"
        uml_code += "activate Subsystem\n\n"

        if status == 'success':
            uml_code += f"Subsystem -> Instrument: Return Success ({start_time}, {end_time})\n"
        else:
            uml_code += f"Subsystem -> Instrument: Return Failure ({start_time}, {end_time})\n"
        
        uml_code += "deactivate Subsystem\n\n"
        uml_code += f"Instrument -> {system_name}: Return Status ({status})\n"
        uml_code += "deactivate Instrument\n\n"
        uml_code += f"{system_name} -> System: Return Status ({status})\n"
        uml_code += f"deactivate {system_name}\n\n"
        uml_code += f"note right of {system_name}: Log Entry - {logtext}\n"

    uml_code += f"System -> User: Return Status ({status})\n"
    uml_code += "deactivate System\n\n"
    uml_code += f"note right of System: Log Entry - {logtext}\n"

    uml_code += "@enduml"
    return uml_code

# Cargar los datos de observación y metadatos
INSTRUMENT = "MATISSE"
RANGE = "1w"
TRACE_ID = 0  # Ejemplo de trace_id, ajusta según tus datos

df_meta = load_meta(INSTRUMENT, RANGE)
df_trace = load_trace(INSTRUMENT, RANGE, TRACE_ID)

# Verificar las columnas de los DataFrames
print("Columnas en df_meta:", df_meta.columns)
print("Columnas en df_trace:", df_trace.columns)

# Seleccionar una fila del DataFrame para el usuario
meta_row = df_meta.iloc[TRACE_ID]

if 'trace_id' in df_trace.columns:
    trace_row = df_trace.iloc[0]
else:
    print("La columna 'trace_id' no está presente en df_trace. Verifica el nombre de la columna.")

# Filtrar y procesar los datos de df_trace
df_trace = load_trace("MATISSE", "1w", 0)
df_trace = df_trace[['@timestamp', 'system', 'procname', 'logtext', 'trace_id']]
df_tratamiento = df_trace[df_trace['logtext'].str.contains("CHKMOON")][:]

lista = ['Send', 'Received command', 'Waiting', 'Succesfully completed', 'Last reply']
Lista_system = []

for x in lista:
    df_cc = df_tratamiento[df_tratamiento['logtext'].str.contains(x)]['system']
    Lista_system.extend(df_cc.tolist())

print(Lista_system)

lista1 = ['Send', 'Received command', 'Waiting', 'Succesfully completed', 'Last reply']
Lista_procname = []

for x in lista1:
    df_cc = df_tratamiento[df_tratamiento['logtext'].str.contains(x)]['procname']
    Lista_procname.extend(df_cc.tolist())

print(Lista_procname)

lista2 = ['Send', 'Received command', 'Waiting', 'Succesfully completed', 'Last reply']
Lista_time = []
for x in lista2:
    df_cc = df_tratamiento[df_tratamiento['logtext'].str.contains(x)]['@timestamp']
    Lista_time.extend(df_cc.tolist())

# Generar el código PlantUML para un solo usuario
uml_code = generate_plantuml_code_single_user(meta_row, trace_row, Lista_system, Lista_procname)

# Guardar el código PlantUML en un archivo
with open('dynamic_diagram.puml', 'w') as file:
    file.write(uml_code)

# Renderizar el diagrama usando PlantUML a través de subprocess
subprocess.run(['java', '-jar', 'plantuml.jar', 'dynamic_diagram.puml'])

Columnas en df_meta: Index(['START', 'END', 'TIMEOUT', 'system', 'procname', 'TPL_ID', 'ERROR',
       'Aborted', 'SECONDS', 'TEL'],
      dtype='object')
Columnas en df_trace: Index(['@timestamp', 'system', 'hostname', 'loghost', 'logtype', 'envname',
       'procname', 'procid', 'module', 'keywname', 'keywvalue', 'keywmask',
       'logtext', 'trace_id'],
      dtype='object')
['MATISSE', 'ISS', 'MATISSE', 'ISS', 'MATISSE']
['bob_ins', 'issifControl', 'bob_ins', 'issifControl', 'bob_ins']


CompletedProcess(args=['java', '-jar', 'plantuml.jar', 'dynamic_diagram.puml'], returncode=1)

In [11]:
lista2 = ['Send', 'Received command', 'Waiting', 'Succesfully completed', 'Last reply']
Lista_time = []
for x in lista2:
    df_cc = df_tratamiento[df_tratamiento['logtext'].str.contains(x)]['@timestamp']
    Lista_procname.extend(df_cc.tolist())
Lista_procname

['bob_ins',
 'issifControl',
 'bob_ins',
 'issifControl',
 'bob_ins',
 Timestamp('2019-04-02 01:30:13.397000'),
 Timestamp('2019-04-02 01:30:13.418000'),
 Timestamp('2019-04-02 01:30:13.419000'),
 Timestamp('2019-04-02 01:30:13.482000'),
 Timestamp('2019-04-02 01:30:14.440000')]

In [12]:
import os
import pandas as pd
import subprocess
from urllib.request import urlretrieve

REPO_URL = 'https://huggingface.co/datasets/Paranal/parlogs-observations/resolve/main/data'

if 'COLAB_RELEASE_TAG' in os.environ.keys():
    PATH = 'data/'  # Convenient name to be Colab compatible
else:
    PATH = 'D:/Ufro/11 nivel/Lab modelacion 2/Trabajo_paramal/collaborations-UFRO-2024-2s/data'  # Local directory to your system

def load_dataset(INSTRUMENT, RANGE):
    fname = f'{INSTRUMENT}-{RANGE}-traces.parket'
    if not os.path.exists(f'{PATH}/{fname}'):
        urlretrieve(f'{REPO_URL}/{fname}', f'{PATH}/{fname}')
    df_inst = pd.read_parquet(f'{PATH}/{fname}')

    fname = f'{INSTRUMENT}-{RANGE}-traces-SUBSYSTEMS.parket'
    if not os.path.exists(f'{PATH}/{fname}'):
        urlretrieve(f'{REPO_URL}/{fname}', f'{PATH}/{fname}')
    df_subs = pd.read_parquet(f'{PATH}/{fname}')

    fname = f'{INSTRUMENT}-{RANGE}-traces-TELESCOPES.parket'
    if not os.path.exists(f'{PATH}/{fname}'):
        urlretrieve(f'{REPO_URL}/{fname}', f'{PATH}/{fname}')
    df_tele = pd.read_parquet(f'{PATH}/{fname}')

    all_traces = [df_inst, df_subs, df_tele]

    df_all = pd.concat(all_traces)
    df_all.sort_values('@timestamp', inplace=True)
    df_all.reset_index(drop=True, inplace=True)

    return df_all

def load_trace(INSTRUMENT, RANGE, trace_id):
    df_all = load_dataset(INSTRUMENT, RANGE)
    df_all = df_all[df_all['trace_id'] == trace_id]
    return df_all

def load_meta(INSTRUMENT, RANGE):
    fname = f'{INSTRUMENT}-{RANGE}-meta.parket'
    if not os.path.exists(f'{PATH}/{fname}'):
        urlretrieve(f'{REPO_URL}/{fname}', f'{PATH}/{fname}')
    df_meta = pd.read_parquet(f'{PATH}/{fname}')
    return df_meta

# Función para generar el código PlantUML para un solo usuario
def generate_plantuml_code_single_user(meta_row, trace_row, Lista_system, Lista_procname, Lista_time):
    uml_code = "@startuml\n"
    uml_code += "actor User\n"
    uml_code += "participant System\n"
    uml_code += "participant Instrument\n"
    uml_code += "participant Subsystem\n\n"

    start_time = meta_row['START']
    end_time = meta_row['END']
    procname = meta_row['procname']
    status = 'success' if not meta_row['ERROR'] else 'failure'
    logtext = trace_row['logtext']

    uml_code += f"User -> System: Send Command ({procname})\n"
    uml_code += "activate System\n"

    previous_procname = None
    for system, procname, proc_time in zip(Lista_system, Lista_procname, Lista_time):
        if procname != previous_procname:
            uml_code += f"note right of System: procname changed to {procname} (Start Time: {proc_time})\n"
            previous_procname = procname

        system_name = "MATISSE" if system == "MATISSE" else "ISS"
        uml_code += f"System -> {system_name}: Forward Command ({procname})\n"
        uml_code += f"activate {system_name}\n"
        uml_code += f"{system_name} -> Instrument: Forward Command ({procname})\n"
        uml_code += "activate Instrument\n"
        uml_code += f"Instrument -> Subsystem: Execute Command ({procname})\n"
        uml_code += "activate Subsystem\n\n"

        if status == 'success':
            uml_code += f"Subsystem -> Instrument: Return Success ({start_time}, {end_time})\n"
        else:
            uml_code += f"Subsystem -> Instrument: Return Failure ({start_time}, {end_time})\n"
        
        uml_code += "deactivate Subsystem\n\n"
        uml_code += f"Instrument -> {system_name}: Return Status ({status})\n"
        uml_code += "deactivate Instrument\n\n"
        uml_code += f"{system_name} -> System: Return Status ({status})\n"
        uml_code += f"deactivate {system_name}\n\n"
        uml_code += f"note right of {system_name}: Log Entry - {logtext}\n"

    uml_code += f"System -> User: Return Status ({status})\n"
    uml_code += "deactivate System\n\n"
    uml_code += f"note right of System: Log Entry - {logtext}\n"

    uml_code += "@enduml"
    return uml_code

# Cargar los datos de observación y metadatos
INSTRUMENT = "MATISSE"
RANGE = "1w"
TRACE_ID = 0  # Ejemplo de trace_id, ajusta según tus datos

df_meta = load_meta(INSTRUMENT, RANGE)
df_trace = load_trace(INSTRUMENT, RANGE, TRACE_ID)

# Verificar las columnas de los DataFrames
print("Columnas en df_meta:", df_meta.columns)
print("Columnas en df_trace:", df_trace.columns)

# Seleccionar una fila del DataFrame para el usuario
meta_row = df_meta.iloc[TRACE_ID]

if 'trace_id' in df_trace.columns:
    trace_row = df_trace.iloc[0]
else:
    print("La columna 'trace_id' no está presente en df_trace. Verifica el nombre de la columna.")

# Filtrar y procesar los datos de df_trace
df_trace = load_trace("MATISSE", "1w", 0)
df_trace = df_trace[['@timestamp', 'system', 'procname', 'logtext', 'trace_id']]
df_tratamiento = df_trace[df_trace['logtext'].str.contains("CHKMOON")][:]

lista = ['Send', 'Received command', 'Waiting', 'Succesfully completed', 'Last reply']
Lista_system = []

for x in lista:
    df_cc = df_tratamiento[df_tratamiento['logtext'].str.contains(x)]['system']
    Lista_system.extend(df_cc.tolist())

print(Lista_system)

lista1 = ['Send', 'Received command', 'Waiting', 'Succesfully completed', 'Last reply']
Lista_procname = []

for x in lista1:
    df_cc = df_tratamiento[df_tratamiento['logtext'].str.contains(x)]['procname']
    Lista_procname.extend(df_cc.tolist())

print(Lista_procname)

lista2 = ['Send', 'Received command', 'Waiting', 'Succesfully completed', 'Last reply']
Lista_time = []
for x in lista2:
    df_cc = df_tratamiento[df_tratamiento['logtext'].str.contains(x)]['@timestamp']
    Lista_time.extend(df_cc.tolist())

print(Lista_time)

# Generar el código PlantUML para un solo usuario
uml_code = generate_plantuml_code_single_user(meta_row, trace_row, Lista_system, Lista_procname, Lista_time)

# Guardar el código PlantUML en un archivo
with open('dynamic_diagram.puml', 'w') as file:
    file.write(uml_code)

# Renderizar el diagrama usando PlantUML a través de subprocess
subprocess.run(['java', '-jar', 'plantuml.jar', 'dynamic_diagram.puml'])

Columnas en df_meta: Index(['START', 'END', 'TIMEOUT', 'system', 'procname', 'TPL_ID', 'ERROR',
       'Aborted', 'SECONDS', 'TEL'],
      dtype='object')
Columnas en df_trace: Index(['@timestamp', 'system', 'hostname', 'loghost', 'logtype', 'envname',
       'procname', 'procid', 'module', 'keywname', 'keywvalue', 'keywmask',
       'logtext', 'trace_id'],
      dtype='object')
['MATISSE', 'ISS', 'MATISSE', 'ISS', 'MATISSE']
['bob_ins', 'issifControl', 'bob_ins', 'issifControl', 'bob_ins']
[Timestamp('2019-04-02 01:30:13.397000'), Timestamp('2019-04-02 01:30:13.418000'), Timestamp('2019-04-02 01:30:13.419000'), Timestamp('2019-04-02 01:30:13.482000'), Timestamp('2019-04-02 01:30:14.440000')]


CompletedProcess(args=['java', '-jar', 'plantuml.jar', 'dynamic_diagram.puml'], returncode=1)

In [13]:
import os
import pandas as pd
import subprocess
from urllib.request import urlretrieve

REPO_URL = 'https://huggingface.co/datasets/Paranal/parlogs-observations/resolve/main/data'

if 'COLAB_RELEASE_TAG' in os.environ.keys():
    PATH = 'data/'  # Convenient name to be Colab compatible
else:
    PATH = 'D:/Ufro/11 nivel/Lab modelacion 2/Trabajo_paramal/collaborations-UFRO-2024-2s/data'  # Local directory to your system

def load_dataset(INSTRUMENT, RANGE):
    fname = f'{INSTRUMENT}-{RANGE}-traces.parket'
    if not os.path.exists(f'{PATH}/{fname}'):
        urlretrieve(f'{REPO_URL}/{fname}', f'{PATH}/{fname}')
    df_inst = pd.read_parquet(f'{PATH}/{fname}')

    fname = f'{INSTRUMENT}-{RANGE}-traces-SUBSYSTEMS.parket'
    if not os.path.exists(f'{PATH}/{fname}'):
        urlretrieve(f'{REPO_URL}/{fname}', f'{PATH}/{fname}')
    df_subs = pd.read_parquet(f'{PATH}/{fname}')

    fname = f'{INSTRUMENT}-{RANGE}-traces-TELESCOPES.parket'
    if not os.path.exists(f'{PATH}/{fname}'):
        urlretrieve(f'{REPO_URL}/{fname}', f'{PATH}/{fname}')
    df_tele = pd.read_parquet(f'{PATH}/{fname}')

    all_traces = [df_inst, df_subs, df_tele]

    df_all = pd.concat(all_traces)
    df_all.sort_values('@timestamp', inplace=True)
    df_all.reset_index(drop=True, inplace=True)

    return df_all

def load_trace(INSTRUMENT, RANGE, trace_id):
    df_all = load_dataset(INSTRUMENT, RANGE)
    df_all = df_all[df_all['trace_id'] == trace_id]
    return df_all

def load_meta(INSTRUMENT, RANGE):
    fname = f'{INSTRUMENT}-{RANGE}-meta.parket'
    if not os.path.exists(f'{PATH}/{fname}'):
        urlretrieve(f'{REPO_URL}/{fname}', f'{PATH}/{fname}')
    df_meta = pd.read_parquet(f'{PATH}/{fname}')
    return df_meta

# Función para generar el código PlantUML para un solo usuario
def generate_plantuml_code_single_user(meta_row, trace_row, Lista_system, Lista_procname, Lista_time):
    uml_code = "@startuml\n"
    uml_code += "actor User\n"
    uml_code += "participant System\n"
    uml_code += "participant Instrument\n"
    uml_code += "participant Subsystem\n\n"

    start_time = meta_row['START']
    end_time = meta_row['END']
    procname = meta_row['procname']
    status = 'success' if not meta_row['ERROR'] else 'failure'
    logtext = trace_row['logtext']

    uml_code += f"User -> System: Send Command ({procname})\n"
    uml_code += "activate System\n"

    previous_procname = None
    for system, procname, proc_time in zip(Lista_system, Lista_procname, Lista_time):
        if procname != previous_procname:
            uml_code += f"note right of System: procname changed to {procname} (Start Time: {proc_time})\n"
            previous_procname = procname

        system_name = "MATISSE" if system == "MATISSE" else "ISS"
        uml_code += f"System -> {system_name}: Forward Command ({procname})\n"
        uml_code += f"activate {system_name}\n"
        uml_code += f"{system_name} -> Instrument: Forward Command ({procname})\n"
        uml_code += "activate Instrument\n"
        uml_code += f"Instrument -> Subsystem: Execute Command ({procname})\n"
        uml_code += "activate Subsystem\n\n"

        if status == 'failure':
            uml_code += f"Subsystem -> Instrument: Return Failure ({start_time}, {end_time})\n"
        
        uml_code += "deactivate Subsystem\n\n"
        uml_code += f"Instrument -> {system_name}: Return Status ({status})\n"
        uml_code += "deactivate Instrument\n\n"
        uml_code += f"{system_name} -> System: Return Status ({status})\n"
        uml_code += f"deactivate {system_name}\n\n"

    uml_code += f"System -> User: Return Status ({status})\n"
    uml_code += "deactivate System\n\n"

    uml_code += "@enduml"
    return uml_code

# Cargar los datos de observación y metadatos
INSTRUMENT = "MATISSE"
RANGE = "1w"
TRACE_ID = 0  # Ejemplo de trace_id, ajusta según tus datos

df_meta = load_meta(INSTRUMENT, RANGE)
df_trace = load_trace(INSTRUMENT, RANGE, TRACE_ID)

# Verificar las columnas de los DataFrames
print("Columnas en df_meta:", df_meta.columns)
print("Columnas en df_trace:", df_trace.columns)

# Seleccionar una fila del DataFrame para el usuario
meta_row = df_meta.iloc[TRACE_ID]

if 'trace_id' in df_trace.columns:
    trace_row = df_trace.iloc[0]
else:
    print("La columna 'trace_id' no está presente en df_trace. Verifica el nombre de la columna.")

# Filtrar y procesar los datos de df_trace
df_trace = load_trace("MATISSE", "1w", 0)
df_trace = df_trace[['@timestamp', 'system', 'procname', 'logtext', 'trace_id']]
df_tratamiento = df_trace[df_trace['logtext'].str.contains("CHKMOON")][:]

lista = ['Send', 'Received command', 'Waiting', 'Succesfully completed', 'Last reply']
Lista_system = []

for x in lista:
    df_cc = df_tratamiento[df_tratamiento['logtext'].str.contains(x)]['system']
    Lista_system.extend(df_cc.tolist())

print(Lista_system)

lista1 = ['Send', 'Received command', 'Waiting', 'Succesfully completed', 'Last reply']
Lista_procname = []

for x in lista1:
    df_cc = df_tratamiento[df_tratamiento['logtext'].str.contains(x)]['procname']
    Lista_procname.extend(df_cc.tolist())

print(Lista_procname)

lista2 = ['Send', 'Received command', 'Waiting', 'Succesfully completed', 'Last reply']
Lista_time = []
for x in lista2:
    df_cc = df_tratamiento[df_tratamiento['logtext'].str.contains(x)]['@timestamp']
    Lista_time.extend(df_cc.tolist())

print(Lista_time)

# Generar el código PlantUML para un solo usuario
uml_code = generate_plantuml_code_single_user(meta_row, trace_row, Lista_system, Lista_procname, Lista_time)

# Guardar el código PlantUML en un archivo
with open('dynamic_diagram.puml', 'w') as file:
    file.write(uml_code)

# Renderizar el diagrama usando PlantUML a través de subprocess
subprocess.run(['java', '-jar', 'plantuml.jar', 'dynamic_diagram.puml'])

Columnas en df_meta: Index(['START', 'END', 'TIMEOUT', 'system', 'procname', 'TPL_ID', 'ERROR',
       'Aborted', 'SECONDS', 'TEL'],
      dtype='object')
Columnas en df_trace: Index(['@timestamp', 'system', 'hostname', 'loghost', 'logtype', 'envname',
       'procname', 'procid', 'module', 'keywname', 'keywvalue', 'keywmask',
       'logtext', 'trace_id'],
      dtype='object')
['MATISSE', 'ISS', 'MATISSE', 'ISS', 'MATISSE']
['bob_ins', 'issifControl', 'bob_ins', 'issifControl', 'bob_ins']
[Timestamp('2019-04-02 01:30:13.397000'), Timestamp('2019-04-02 01:30:13.418000'), Timestamp('2019-04-02 01:30:13.419000'), Timestamp('2019-04-02 01:30:13.482000'), Timestamp('2019-04-02 01:30:14.440000')]


CompletedProcess(args=['java', '-jar', 'plantuml.jar', 'dynamic_diagram.puml'], returncode=1)

In [14]:
import os
import pandas as pd
import subprocess
from urllib.request import urlretrieve

REPO_URL = 'https://huggingface.co/datasets/Paranal/parlogs-observations/resolve/main/data'

if 'COLAB_RELEASE_TAG' in os.environ.keys():
    PATH = 'data/'  # Convenient name to be Colab compatible
else:
    PATH = 'D:/Ufro/11 nivel/Lab modelacion 2/Trabajo_paramal/collaborations-UFRO-2024-2s/data'  # Local directory to your system

def load_dataset(INSTRUMENT, RANGE):
    fname = f'{INSTRUMENT}-{RANGE}-traces.parket'
    if not os.path.exists(f'{PATH}/{fname}'):
        urlretrieve(f'{REPO_URL}/{fname}', f'{PATH}/{fname}')
    df_inst = pd.read_parquet(f'{PATH}/{fname}')

    fname = f'{INSTRUMENT}-{RANGE}-traces-SUBSYSTEMS.parket'
    if not os.path.exists(f'{PATH}/{fname}'):
        urlretrieve(f'{REPO_URL}/{fname}', f'{PATH}/{fname}')
    df_subs = pd.read_parquet(f'{PATH}/{fname}')

    fname = f'{INSTRUMENT}-{RANGE}-traces-TELESCOPES.parket'
    if not os.path.exists(f'{PATH}/{fname}'):
        urlretrieve(f'{REPO_URL}/{fname}', f'{PATH}/{fname}')
    df_tele = pd.read_parquet(f'{PATH}/{fname}')

    all_traces = [df_inst, df_subs, df_tele]

    df_all = pd.concat(all_traces)
    df_all.sort_values('@timestamp', inplace=True)
    df_all.reset_index(drop=True, inplace=True)

    return df_all

def load_trace(INSTRUMENT, RANGE, trace_id):
    df_all = load_dataset(INSTRUMENT, RANGE)
    df_all = df_all[df_all['trace_id'] == trace_id]
    return df_all

def load_meta(INSTRUMENT, RANGE):
    fname = f'{INSTRUMENT}-{RANGE}-meta.parket'
    if not os.path.exists(f'{PATH}/{fname}'):
        urlretrieve(f'{REPO_URL}/{fname}', f'{PATH}/{fname}')
    df_meta = pd.read_parquet(f'{PATH}/{fname}')
    return df_meta

# Función para generar el código PlantUML para un solo usuario
def generate_plantuml_code_single_user(meta_row, trace_row, Lista_system, Lista_procname, Lista_time):
    uml_code = "@startuml\n"
    uml_code += "actor User\n"
    uml_code += "participant System\n"
    uml_code += "participant Instrument\n"
    uml_code += "participant Subsystem\n\n"

    start_time = meta_row['START']
    end_time = meta_row['END']
    procname = meta_row['procname']
    status = 'success' if not meta_row['ERROR'] else 'failure'
    logtext = trace_row['logtext']

    uml_code += "User -> System: Send Command CHKMOON\n"
    uml_code += "activate System\n"

    previous_procname = None
    for system, procname, proc_time in zip(Lista_system, Lista_procname, Lista_time):
        if procname != previous_procname:
            # Calcular la duración del procname
            proc_start_time = pd.to_datetime(proc_time)
            proc_end_time = pd.to_datetime(proc_time)  # Ajusta esto según tus datos
            proc_duration = proc_end_time - proc_start_time
            uml_code += f"note right of System: procname changed to {procname} (Start Time: {proc_time}, Duration: {proc_duration})\n"
            previous_procname = procname

        system_name = "MATISSE" if system == "MATISSE" else "ISS"
        uml_code += f"System -> {system_name}: Forward Command ({procname})\n"
        uml_code += f"activate {system_name}\n"
        uml_code += f"{system_name} -> Instrument: Forward Command ({procname})\n"
        uml_code += "activate Instrument\n"
        uml_code += f"Instrument -> Subsystem: Execute Command ({procname})\n"
        uml_code += "activate Subsystem\n\n"

        if status == 'success':
            uml_code += "Subsystem -> Instrument: Return Success\n"
        else:
            uml_code += f"Subsystem -> Instrument: Return Failure ({start_time}, {end_time})\n"
        
        uml_code += "deactivate Subsystem\n\n"
        uml_code += f"Instrument -> {system_name}: Return Status ({status})\n"
        uml_code += "deactivate Instrument\n\n"
        uml_code += f"{system_name} -> System: Return Status ({status})\n"
        uml_code += f"deactivate {system_name}\n\n"

    uml_code += f"System -> User: Return Status ({status})\n"
    uml_code += "deactivate System\n\n"

    uml_code += "@enduml"
    return uml_code

# Cargar los datos de observación y metadatos
INSTRUMENT = "MATISSE"
RANGE = "1w"
TRACE_ID = 0  # Ejemplo de trace_id, ajusta según tus datos

df_meta = load_meta(INSTRUMENT, RANGE)
df_trace = load_trace(INSTRUMENT, RANGE, TRACE_ID)

# Verificar las columnas de los DataFrames
print("Columnas en df_meta:", df_meta.columns)
print("Columnas en df_trace:", df_trace.columns)

# Seleccionar una fila del DataFrame para el usuario
meta_row = df_meta.iloc[TRACE_ID]

if 'trace_id' in df_trace.columns:
    trace_row = df_trace.iloc[0]
else:
    print("La columna 'trace_id' no está presente en df_trace. Verifica el nombre de la columna.")

# Filtrar y procesar los datos de df_trace
df_trace = load_trace("MATISSE", "1w", 0)
df_trace = df_trace[['@timestamp', 'system', 'procname', 'logtext', 'trace_id']]
df_tratamiento = df_trace[df_trace['logtext'].str.contains("CHKMOON")][:]

lista = ['Send', 'Received command', 'Waiting', 'Succesfully completed', 'Last reply']
Lista_system = []

for x in lista:
    df_cc = df_tratamiento[df_tratamiento['logtext'].str.contains(x)]['system']
    Lista_system.extend(df_cc.tolist())

print(Lista_system)

lista1 = ['Send', 'Received command', 'Waiting', 'Succesfully completed', 'Last reply']
Lista_procname = []

for x in lista1:
    df_cc = df_tratamiento[df_tratamiento['logtext'].str.contains(x)]['procname']
    Lista_procname.extend(df_cc.tolist())

print(Lista_procname)

lista2 = ['Send', 'Received command', 'Waiting', 'Succesfully completed', 'Last reply']
Lista_time = []
for x in lista2:
    df_cc = df_tratamiento[df_tratamiento['logtext'].str.contains(x)]['@timestamp']
    Lista_time.extend(df_cc.tolist())

print(Lista_time)

# Generar el código PlantUML para un solo usuario
uml_code = generate_plantuml_code_single_user(meta_row, trace_row, Lista_system, Lista_procname, Lista_time)

# Guardar el código PlantUML en un archivo
with open('dynamic_diagram.puml', 'w') as file:
    file.write(uml_code)

# Renderizar el diagrama usando PlantUML a través de subprocess
subprocess.run(['java', '-jar', 'plantuml.jar', 'dynamic_diagram.puml'])

Columnas en df_meta: Index(['START', 'END', 'TIMEOUT', 'system', 'procname', 'TPL_ID', 'ERROR',
       'Aborted', 'SECONDS', 'TEL'],
      dtype='object')
Columnas en df_trace: Index(['@timestamp', 'system', 'hostname', 'loghost', 'logtype', 'envname',
       'procname', 'procid', 'module', 'keywname', 'keywvalue', 'keywmask',
       'logtext', 'trace_id'],
      dtype='object')
['MATISSE', 'ISS', 'MATISSE', 'ISS', 'MATISSE']
['bob_ins', 'issifControl', 'bob_ins', 'issifControl', 'bob_ins']
[Timestamp('2019-04-02 01:30:13.397000'), Timestamp('2019-04-02 01:30:13.418000'), Timestamp('2019-04-02 01:30:13.419000'), Timestamp('2019-04-02 01:30:13.482000'), Timestamp('2019-04-02 01:30:14.440000')]


CompletedProcess(args=['java', '-jar', 'plantuml.jar', 'dynamic_diagram.puml'], returncode=1)

# Secuencia de codigo utilizado para el codigo UML

In [15]:
df_meta = load_meta("MATISSE", "1w")
df_meta

,START,END,TIMEOUT,system,procname,TPL_ID,ERROR,Aborted,SECONDS,TEL
0,2019-04-02 01:29:45.681,2019-04-02 01:38:08.541,False,MATISSE,bob_ins,MATISSE_img_acq,False,False,502.0,AT
1,2019-04-02 01:38:08.559,2019-04-02 01:44:02.506,False,MATISSE,bob_ins,MATISSE_hyb_obs,False,False,353.0,AT
2,2019-04-02 01:45:44.711,2019-04-02 01:58:00.921,False,MATISSE,bob_ins,MATISSE_img_acq,False,False,736.0,AT
3,2019-04-02 01:58:19.863,2019-04-02 02:14:00.315,False,MATISSE,bob_ins,MATISSE_hyb_obs,False,False,940.0,AT
4,2019-04-02 06:36:28.631,2019-04-02 07:00:16.566,False,MATISSE,bob_ins,MATISSE_img_acq,False,False,1427.0,AT
...,...,...,...,...,...,...,...,...,...,...
215,2019-04-07 21:20:07.817,2019-04-07 21:22:12.808,False,MATISSE,bob_ins,MATISSE_gen_tec_opd,False,False,124.0,AT
216,2019-04-07 21:22:25.249,2019-04-07 21:34:42.495,False,MATISSE,bob_ins,MATISSE_gen_tec_check_fringes,False,False,737.0,AT
217,2019-04-07 21:34:42.495,2019-04-07 21:39:06.173,False,MATISSE,bob_ins,MATISSE_gen_cal_bias,False,False,263.0,AT
218,2019-04-07 21:40:15.747,2019-04-07 21:42:02.235,False,MATISSE,bob_ins,MATISSE_gen_cal_bias,False,False,106.0,AT


In [16]:
df_trace = load_trace("MATISSE", "1w", 0)
df_trace = df_trace[['@timestamp', 'system', 'procname', 'logtext', 'trace_id']]

In [17]:
with pd.option_context('display.max_colwidth', None):
    display(df_trace[ df_trace['logtext'].str.contains("CHKMOON") ][:5])

,@timestamp,system,procname,logtext,trace_id
182,2019-04-02 01:30:13.397,MATISSE,bob_ins,"Send command CHKMOON 064508.910,-164258.000,0.0 to process issifControl env. wvgvlti",0
183,2019-04-02 01:30:13.418,ISS,issifControl,"CDP : Received command: CHKMOON, Buffer: 064508.910,-164258.000,0.0",0
184,2019-04-02 01:30:13.419,MATISSE,bob_ins,Waiting (timeout 40000) for pending replies to command CHKMOON id cmd0,0
185,2019-04-02 01:30:13.482,ISS,issifControl,"CDP : Succesfully completed command: CHKMOON, Buffer: OK",0
194,2019-04-02 01:30:14.440,MATISSE,bob_ins,Last reply to command CHKMOON id cmd0 received,0
